<a href="https://colab.research.google.com/github/mahesa005/Data-Mining---Action-2025/blob/main/action-colabnb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import os
import random

import numpy as np
import torch


def seed_everything(seed: int = 42) -> None:
    """Seed semua sumber acak untuk hasil yang reprodusibel."""
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


SEED = 42
seed_everything(SEED)
print(f"Reproducibility seed set to {SEED}")

Reproducibility seed set to 42


In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
# Copyright 2018 Google Inc.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
"""Colab-specific Google Drive integration."""

import collections as _collections
import os as _os
import signal as _signal
import socket as _socket
import subprocess as _subprocess
import sys as _sys
import uuid as _uuid

from google.colab import _message
from google.colab import output as _output

import pexpect.popen_spawn as _popen_spawn
import psutil as _psutil

__all__ = ['flush_and_unmount', 'mount']

_Environment = _collections.namedtuple(
    '_Environment',
    ('home', 'root_dir', 'dev', 'path', 'config_dir'),
)


def _env():
  """Create and return an _Environment to use."""
  home = _os.environ['HOME']
  root_dir = _os.path.realpath(
      _os.path.join(_os.environ['CLOUDSDK_CONFIG'], '../..')
  )
  dev = '/dev/fuse'
  path = '/usr/local/bin:/usr/local/sbin:/usr/bin:/usr/sbin:/bin:/sbin:.'
  if len(root_dir) > 1 and not root_dir.startswith('/usr/local/google/'):
    home = _os.path.join(root_dir, home)
    fum = _os.environ['HOME'].split('mount')[0] + '/mount/alloc/fusermount'
    dev = fum + '/dev/fuse'
    path = path + ':' + fum + '/bin'
  config_dir = _os.path.join(home, '.config', 'Google')
  return _Environment(
      home=home,
      root_dir=root_dir,
      dev=dev,
      path=path,
      config_dir=config_dir,
  )


def _logs_dir():
  return _os.path.join(_env().config_dir, 'DriveFS/Logs/')


def _timeouts_path():
  return _os.path.join(_logs_dir(), 'timeouts.txt')


def flush_and_unmount(timeout_ms=24 * 60 * 60 * 1000):
  """Unmount Google Drive and flush any outstanding writes to it."""
  if _os.path.exists('/var/colab/mp'):
    raise NotImplementedError(__name__ + ' is unsupported in this environment.')
  env = _env()
  if b'type fuse.drive' not in _subprocess.check_output(['/bin/mount']):
    print('Drive not mounted, so nothing to flush and unmount.')
    return
  drive_bin = _os.path.join(env.root_dir, 'opt/google/drive/drive')
  p = _subprocess.Popen(
      [
          drive_bin,
          '--push_changes_and_quit',
          '--single_process',
          '--timeout_sec={}'.format(int(timeout_ms / 1000)),
      ],
      stdout=_subprocess.PIPE,
      stderr=_subprocess.PIPE,
  )
  out, err = p.communicate()
  if mount._DEBUG:  # pylint:disable=protected-access
    print('flush_and_unmount: out: {}\nerr: {}'.format(out, err))
  if p.returncode:
    raise ValueError('flush_and_unmount failed')


def mount(mountpoint, force_remount=False, timeout_ms=120000, readonly=False):
  """Mount your Google Drive at the specified mountpoint path."""
  return _mount(
      mountpoint,
      force_remount=force_remount,
      timeout_ms=timeout_ms,
      ephemeral=True,
      readonly=readonly,
  )


def _mount(
    mountpoint,
    force_remount=False,
    timeout_ms=120000,
    ephemeral=False,
    readonly=False,
):
  """Internal helper to mount Google Drive."""
  if not _os.path.exists('/var/colab/hostname'):
    raise NotImplementedError(
        'Mounting drive is unsupported in this environment. Use PyDrive2'
        ' instead. See examples at'
        ' https://colab.research.google.com/notebooks/io.ipynb#scrollTo=7taylj9wpsA2.'
    )

  if ' ' in mountpoint:
    raise ValueError('Mountpoint must not contain a space.')

  if _os.environ.get('VERTEX_PRODUCT') == 'COLAB_ENTERPRISE':
    raise NotImplementedError(
        'google.colab.drive.mount is not supported in Colab Enterprise.'
    )
  metadata_server_addr = (
      _os.environ['TBE_EPHEM_CREDS_ADDR']
      if ephemeral
      else _os.environ['TBE_CREDS_ADDR']
  )
  if ephemeral:
    _message.blocking_request(
        'request_auth',
        request={'authType': 'dfs_ephemeral'},
        timeout_sec=int(timeout_ms / 1000),
    )

  mountpoint = _os.path.expanduser(mountpoint)
  # If we've already mounted drive at the specified mountpoint, exit now.
  already_mounted = _os.path.isdir(_os.path.join(mountpoint, 'My Drive'))
  if not force_remount and already_mounted:
    print(
        f'Drive already mounted at {mountpoint}; to attempt to forcibly '
        f'remount, call drive.mount("{mountpoint}", force_remount=True).'
    )
    return

  env = _env()
  home = env.home
  root_dir = env.root_dir
  dev = env.dev
  path = env.path
  config_dir = env.config_dir

  try:
    _os.makedirs(config_dir)
  except OSError:
    if not _os.path.isdir(config_dir):
      raise ValueError(f'{config_dir} must be a directory if present')  # pylint: disable=raise-missing-from

  # Launch an intermediate bash to manage DriveFS' I/O (b/141747058#comment6).
  prompt = 'root@{}-{}: '.format(_socket.gethostname(), _uuid.uuid4().hex)
  logfile = None
  if mount._DEBUG:  # pylint:disable=protected-access
    logfile = _sys.stdout
  d = _popen_spawn.PopenSpawn(
      '/usr/bin/setsid /bin/bash --noediting -i',  # Need -i to get prompt echo.
      # Pad in order to more reliably detect the `timeout_pattern` case below.
      timeout=int(timeout_ms / 1000 + 30),
      maxread=int(1e6),
      encoding='utf-8',
      logfile=logfile,
      env={'HOME': home, 'FUSE_DEV_NAME': dev, 'PATH': path},
  )
  d.sendline(f'unset HISTFILE; export PS1="{prompt}"')
  d.expect(prompt)  # The new prompt.
  drive_dir = _os.path.join(root_dir, 'opt/google/drive')
  # Robustify to previously-running copies of drive. Don't only [pkill -9]
  # because that leaves enough cruft behind in the mount table that future
  # operations fail with "Transport endpoint is not connected".
  d.sendline(
      f'umount -f {mountpoint} || umount {mountpoint}; pkill -9 -x drive'
  )
  # Wait for above to be received, using the next prompt.
  d.expect(prompt)
  d.sendline(f'pkill -9 -f {drive_dir}/directoryprefetcher_binary')
  d.expect(prompt)
  # Only check the mountpoint after potentially unmounting/pkill'ing above.
  try:
    if _os.path.islink(mountpoint):
      raise ValueError('Mountpoint must not be a symlink')
    if _os.path.isdir(mountpoint) and _os.listdir(mountpoint):
      raise ValueError('Mountpoint must not already contain files')
    if not _os.path.isdir(mountpoint) and _os.path.exists(mountpoint):
      raise ValueError('Mountpoint must either be a directory or not exist')
    normed = _os.path.normpath(mountpoint)
    if '/' in normed and not _os.path.exists(_os.path.dirname(normed)):
      raise ValueError('Mountpoint must be in a directory that exists')
  except:
    d.kill(_signal.SIGKILL)
    raise

  # Watch for success.
  success = 'google.colab.drive MOUNTED'
  success_watcher = (
      '( while `sleep 0.5`; do if [[ -d "{m}" && "$(ls -A {m})" != "" ]]; '
      'then echo "{s}"; break; fi; done ) &'
  ).format(m=mountpoint, s=success)
  d.sendline(success_watcher)
  d.expect(prompt)

  domain_disabled_drivefs = 'The domain policy has disabled Drive File Stream'
  problem_and_stopped = (
      'Drive File Stream encountered a problem and has stopped'
  )
  drive_exited = 'drive EXITED'

  d.sendline(
      '('
      f' {drive_dir}/drive'
      ' --features='
      'crash_throttle_percentage:100,'
      'fuse_max_background:1000,'
      'max_read_qps:1000,'
      'max_write_qps:1000,'
      'max_operation_batch_size:15,'
      'max_parallel_push_task_instances:10,'
      f'opendir_timeout_ms:{timeout_ms},'
      'virtual_folders_omit_spaces:true,'
      f'read_only_mode:{str(readonly).lower()}'
      f' --metadata_server_auth_uri={metadata_server_addr}/computeMetadata/v1'
      ' --preferences='
      f'trusted_root_certs_file_path:{drive_dir}/roots.pem,'
      'feature_flag_restart_seconds:129600,'
      f'mount_point_path:{mountpoint}'
      ' 2>&1 |'
      ' grep --line-buffered -E'
      f' "{problem_and_stopped}|{domain_disabled_drivefs}";'
      f' echo "{drive_exited}";'
      ') &'
  )
  d.expect(prompt)

  # LINT.IfChange(drivetimeout)
  timeout_pattern = 'QueryManager timed out'
  # LINT.ThenChange()
  dfs_log = _os.path.join(_logs_dir(), 'drive_fs.txt')

  while True:
    case = d.expect([
        success,
        prompt,
        problem_and_stopped,
        drive_exited,
        domain_disabled_drivefs,
    ])
    if case == 0:
      break
    elif case == 1 or case == 2 or case == 3:
      # Prompt appearing here means something went wrong with the drive binary.
      d.kill(_signal.SIGKILL)
      extra_reason = ''
      if 0 == _subprocess.call(
          f'grep -q "{timeout_pattern}" "{dfs_log}"', shell=True
      ):
        extra_reason = (
            ': timeout during initial read of root folder; for more info: '
            'https://research.google.com/colaboratory/faq.html#drive-timeout'
        )
      raise ValueError('mount failed' + extra_reason)
    elif case == 4:
      # Terminate the DriveFS binary before killing bash.
      for p in _psutil.process_iter():
        if p.name() == 'drive':
          p.kill()
      # Now kill bash.
      d.kill(_signal.SIGKILL)
      raise ValueError(
          str(domain_disabled_drivefs)
          + ': https://support.google.com/a/answer/7496409'
      )
  filtered_logfile = _timeouts_path()
  d.sendline('fuser -kw "{f}" ; rm -rf "{f}"'.format(f=filtered_logfile))
  d.expect(prompt)
  filter_script = _os.path.join(drive_dir, 'drive-filter.py')
  filter_cmd = (
      """nohup bash -c 'tail -n +0 -F "{}" | """
      """python3 {} > "{}" ' < /dev/null > /dev/null 2>&1 &"""
  ).format(dfs_log, filter_script, filtered_logfile)
  d.sendline(filter_cmd)
  d.expect(prompt)
  if 'ENABLE_DIRECTORYPREFETCHER' in _os.environ:
    d.sendline(
        """nohup bash -c '{d}/directoryprefetcher_binary -mountpoint={mnt}' """
        """>> {log} 2>&1 &""".format(
            d=drive_dir,
            mnt=mountpoint,
            log=_os.path.join(_logs_dir(), 'dpb.txt'),
        )
    )
    d.expect(prompt)
  d.sendline('disown -a')
  d.expect(prompt)
  d.sendline('exit')
  assert d.wait() == 0
  _output.clear(wait=True, output_tags='dfs-auth-dance')
  print(f'Mounted at {mountpoint}')


mount._DEBUG = False  # pylint:disable=protected-access


In [11]:
!pip install imagehash

In [12]:
import os
from contextlib import nullcontext
from pathlib import Path

import cv2
import imagehash
import numpy as np
import pandas as pd
import timm
import torch
import torch.nn as nn
import torch.optim as optim
from PIL import Image
from sklearn.metrics import classification_report, f1_score
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from tqdm.auto import tqdm

# ===================================================================
# === 2. KONFIGURASI DAN SETUP ===
# ===================================================================

BASE_DIR = '/content/drive/My Drive/ACTION' # Menggunakan path yang disediakan user
TRAIN_DIR = os.path.join(BASE_DIR, "dataset fixx", "train")
TEST_DIR = os.path.join(BASE_DIR, "dataset fixx", "test", "test")

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
AMP_ENABLED = DEVICE.type == "cuda"
IMG_SIZE = 224 # DIUBAH DARI 192 MENJADI 224 UNTUK MENYESUAIKAN MODEL
BATCH_SIZE = 8
NUM_WORKERS = os.cpu_count() or 1

EPOCHS_S1 = 2
LR_S1 = 3e-4
EPOCHS_S2 = 3
LR_S2 = 1e-4
EPOCHS_S3 = 6
LR_S3 = 5e-5

IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD = [0.229, 0.224, 0.225]
BEST_MODEL_PATH = "best_swin_model.pth"

print(f"Menggunakan data latih dari: {TRAIN_DIR}")
print(f"Menggunakan data uji dari: {TEST_DIR}")
print(f"Menggunakan device: {DEVICE}")
print(f"Ukuran gambar: {IMG_SIZE}x{IMG_SIZE}, Batch Size: {BATCH_SIZE}")

autocast_ctx = torch.cuda.amp.autocast if AMP_ENABLED else nullcontext
scaler = torch.cuda.amp.GradScaler(enabled=AMP_ENABLED)

Menggunakan data latih dari: /content/drive/My Drive/ACTION/dataset fixx/train
Menggunakan data uji dari: /content/drive/My Drive/ACTION/dataset fixx/test/test
Menggunakan device: cuda
Ukuran gambar: 224x224, Batch Size: 8


/tmp/ipython-input-848585201.py:52: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=AMP_ENABLED)


In [13]:
# ===================================================================
# === 3. FUNGSI DAN KELAS HELPER ===
# ===================================================================

def convert_path_to_df(dataset, is_test=False):
    image_dir = Path(dataset)
    filepaths = list(image_dir.glob(r"**/*.*"))
    if not filepaths:
        raise FileNotFoundError(f"Tidak menemukan berkas gambar di direktori {image_dir.resolve()}")
    if is_test:
        filepaths = pd.Series(filepaths, name="Filepath").astype(str)
        return pd.DataFrame({"Filepath": filepaths})
    labels = [p.parts[-2] for p in filepaths]
    filepaths = pd.Series(filepaths, name="Filepath").astype(str)
    labels = pd.Series(labels, name="Label")
    return pd.concat([filepaths, labels], axis=1)


class CustomDataset(Dataset):
    def __init__(self, dataframe, image_column, label_column=None, transform=None):
        self.dataframe = dataframe
        self.image_column = image_column
        self.label_column = label_column
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = self.dataframe.iloc[idx][self.image_column]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        if self.label_column is None:
            return image
        label = self.dataframe.iloc[idx][self.label_column]
        return image, torch.tensor(label, dtype=torch.long)


class DualTransformDataset(Dataset):
    def __init__(self, dataframe, image_column, label_column, transform_main, transform_extra):
        self.dataframe = dataframe
        self.image_column = image_column
        self.label_column = label_column
        self.transform_main = transform_main
        self.transform_extra = transform_extra

    def __len__(self):
        return len(self.dataframe) * 2

    def __getitem__(self, idx):
        base_idx = idx // 2
        use_extra = idx % 2 == 1
        row = self.dataframe.iloc[base_idx]
        img_path = row[self.image_column]
        image = Image.open(img_path).convert("RGB")
        image = self.transform_extra(image) if use_extra else self.transform_main(image)
        label = torch.tensor(row[self.label_column], dtype=torch.long)
        return image, label


class CLAHETransform:
    def __init__(self, clip_limit=2.0, tile_grid_size=(8, 8)):
        self.clip_limit = clip_limit
        self.tile_grid_size = tile_grid_size

    def __call__(self, img):
        img_np = np.array(img)
        lab = cv2.cvtColor(img_np, cv2.COLOR_RGB2LAB)
        l, a, b = cv2.split(lab)
        clahe = cv2.createCLAHE(clipLimit=self.clip_limit, tileGridSize=self.tile_grid_size)
        l = clahe.apply(l)
        lab = cv2.merge((l, a, b))
        img_rgb_clahe = cv2.cvtColor(lab, cv2.COLOR_LAB2RGB)
        return Image.fromarray(img_rgb_clahe)


class TestDataset(Dataset):
    def __init__(self, dataframe, image_column, transform=None):
        self.dataframe = dataframe
        self.image_column = image_column
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = self.dataframe.iloc[idx][self.image_column]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, img_path


def get_phash(filepath):
    try:
        with Image.open(filepath) as img:
            return imagehash.phash(img)
    except Exception:
        return None

In [7]:
# ===================================================================
# === 4. PERSIAPAN DATA DENGAN STRATEGI "VALIDASI BERSIH" ===
# ===================================================================

train_df = convert_path_to_df(TRAIN_DIR)
print(f"Jumlah data training sebelum filtering manual: {len(train_df)}")

print()
print("=" * 50)
print("Mengidentifikasi kebocoran data (train vs test)...")

test_df = convert_path_to_df(TEST_DIR, is_test=True)
tqdm.pandas(desc="Menghitung Hash Data Test")
test_hashes = set(test_df["Filepath"].progress_apply(get_phash))
test_hashes.discard(None)

tqdm.pandas(desc="Menghitung Hash Data Train")
train_df["hash"] = train_df["Filepath"].progress_apply(get_phash)
train_df["is_leak"] = train_df["hash"].isin(test_hashes)
print(f"Ditemukan {train_df['is_leak'].sum()} gambar di training set yang identik dengan gambar di test set.")

print()
print("Menerapkan strategi 'Validasi Bersih'...")
leaked_df = train_df[train_df["is_leak"]].copy()
clean_df = train_df[~train_df["is_leak"]].copy()

label_mapping = {
    "Ayam Bakar": 0,
    "Ayam Betutu": 1,
    "Ayam Goreng": 2,
    "Ayam Pop": 3,
    "Bakso": 4,
    "Coto Makassar": 5,
    "Gado Gado": 6,
    "Gudeg": 7,
    "Nasi Goreng": 8,
    "Pempek": 9,
    "Rawon": 10,
    "Rendang": 11,
    "Sate Madura": 12,
    "Sate Padang": 13,
    "Soto": 14,
}

clean_df["Label"] = clean_df["Label"].map(label_mapping)
leaked_df["Label"] = leaked_df["Label"].map(label_mapping)

val_split = pd.DataFrame()
if not clean_df.empty:
    try:
        clean_train_split, val_split = train_test_split(
            clean_df, test_size=0.2, random_state=42, stratify=clean_df["Label"]
        )
    except ValueError:
        print("Peringatan: Gagal stratify, menggunakan split biasa.")
        clean_train_split, val_split = train_test_split(clean_df, test_size=0.2, random_state=42)
else:
    clean_train_split = clean_df

train_split = pd.concat([clean_train_split, leaked_df], ignore_index=True)
train_split.drop(columns=["hash", "is_leak"], inplace=True, errors="ignore")

train_split["Label"] = train_split["Label"].astype(int)
if "Label" in val_split.columns:
    val_split["Label"] = val_split["Label"].astype(int)

print(f"Total data train setelah digabung: {len(train_split)}")
print(f"Total data validasi bersih: {len(val_split)}")

Jumlah data training sebelum filtering manual: 4052

Mengidentifikasi kebocoran data (train vs test)...


Menghitung Hash Data Test:   0%|          | 0/2057 [00:00<?, ?it/s]

Menghitung Hash Data Train:   0%|          | 0/4052 [00:00<?, ?it/s]

Ditemukan 80 gambar di training set yang identik dengan gambar di test set.

Menerapkan strategi 'Validasi Bersih'...
Total data train setelah digabung: 3257
Total data validasi bersih: 795


In [14]:
# ===================================================================
# === 5. TRANSFORMASI DATA ===
# ===================================================================

train_transform = transforms.Compose([
    CLAHETransform(),
    transforms.RandomResizedCrop(IMG_SIZE, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.2),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomApply([transforms.GaussianBlur(kernel_size=3)], p=0.3),
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),
])

train_transform_extra = transforms.Compose([
    CLAHETransform(),
    transforms.RandomRotation(degrees=15),
    transforms.RandomResizedCrop(IMG_SIZE, scale=(0.6, 1.0)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),
])

val_test_transform = transforms.Compose([
    CLAHETransform(),
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),
])

train_dataset = DualTransformDataset(
    train_split,
    image_column="Filepath",
    label_column="Label",
    transform_main=train_transform,
    transform_extra=train_transform_extra,
)

val_dataset = CustomDataset(val_split, "Filepath", "Label", val_test_transform)

train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=NUM_WORKERS,
    persistent_workers=NUM_WORKERS > 0,
)

val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS,
    persistent_workers=NUM_WORKERS > 0,
)

class_weights = compute_class_weight(
    "balanced", classes=np.unique(train_split["Label"]), y=train_split["Label"]
)
class_weights = torch.tensor(class_weights, dtype=torch.float32, device=DEVICE)

In [15]:
# ===================================================================
# === 6. DEFINISI MODEL SWIN TRANSFORMER ===
# ===================================================================


class SingleSwinModel(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.backbone = timm.create_model(
            "swin_base_patch4_window7_224", pretrained=True, num_classes=num_classes
        )

    def forward(self, x):
        return self.backbone(x)

    def freeze_all(self):
        for param in self.backbone.parameters():
            param.requires_grad = False

    def enable_head(self):
        for name, param in self.backbone.named_parameters():
            if "head" in name or "norm" in name:
                param.requires_grad = True

    def enable_stages(self, stages):
        for name, param in self.backbone.named_parameters():
            if any(f"layers.{idx}" in name for idx in stages):
                param.requires_grad = True

    def enable_all(self):
        for param in self.backbone.parameters():
            param.requires_grad = True


model = SingleSwinModel(num_classes=len(label_mapping)).to(DEVICE)
model.freeze_all()
model.enable_head()

criterion = nn.CrossEntropyLoss(weight=class_weights)


def train_one_epoch(model, loader, optimizer, criterion, scheduler=None):
    model.train()
    running_loss = 0.0
    for images, labels in tqdm(loader, desc="Train", leave=False):
        images = images.to(DEVICE)
        labels = labels.to(DEVICE)
        optimizer.zero_grad(set_to_none=True);
        with autocast_ctx():
            outputs = model(images)
            loss = criterion(outputs, labels)
        scaler.scale(loss).backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        scaler.step(optimizer)
        scaler.update()
        if scheduler is not None:
            scheduler.step()
        running_loss += loss.item()
    return running_loss / max(1, len(loader))


def evaluate(model, loader, criterion):
    if len(getattr(loader, "dataset", [])) == 0:
        return 0.0, 0.0, "Validasi kosong - dilewati."
    model.eval()
    running_loss = 0.0
    all_preds, all_labels = [], []
    with torch.no_grad():
        for images, labels in tqdm(loader, desc="Val", leave=False):
            images = images.to(DEVICE)
            labels = labels.to(DEVICE)
            with autocast_ctx():
                outputs = model(images)
                loss = criterion(outputs, labels)
            running_loss += loss.item()
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    avg_loss = running_loss / max(1, len(loader))
    f1 = f1_score(all_labels, all_preds, average="macro")
    report = classification_report(all_labels, all_preds, target_names=list(label_mapping.keys()))
    return avg_loss, f1, report


best_f1 = 0.0

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

In [16]:
print()
print("=" * 50)
print("TAHAP 1: Fine-tuning Head Klasifikasi")
print("=" * 50)
optimizer = optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=LR_S1, weight_decay=1e-4)
for epoch in range(EPOCHS_S1):
    train_loss = train_one_epoch(model, train_loader, optimizer, criterion)
    val_loss, f1, report = evaluate(model, val_loader, criterion)
    print(f"Epoch [{epoch + 1}/{EPOCHS_S1}] | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Macro F1: {f1:.4f}")
    print("--- Laporan Klasifikasi Validasi ---")
    print(report)
    if f1 > best_f1:
        best_f1 = f1
        torch.save(model.state_dict(), BEST_MODEL_PATH)
        print(f"✅ Model disimpan (F1 terbaik baru: {best_f1:.4f})")


TAHAP 1: Fine-tuning Head Klasifikasi


Train:   0%|          | 0/815 [00:00<?, ?it/s]

/tmp/ipython-input-845850214.py:49: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast_ctx():


Val:   0%|          | 0/100 [00:00<?, ?it/s]

/tmp/ipython-input-845850214.py:72: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast_ctx():


...

Train:   0%|          | 0/815 [00:00<?, ?it/s]

/tmp/ipython-input-845850214.py:49: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast_ctx():


Val:   0%|          | 0/100 [00:00<?, ?it/s]

/tmp/ipython-input-845850214.py:72: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast_ctx():


Epoch [2/2] | Train Loss: 0.3291 | Val Loss: 0.3252 | Macro F1: 0.9133
--- Laporan Klasifikasi Validasi ---
               precision    recall  f1-score   support

   Ayam Bakar       0.90      0.78      0.83        55
  Ayam Betutu       0.85      0.78      0.82        51
  Ayam Goreng       0.69      0.89      0.77        54
     Ayam Pop       0.94      0.94      0.94        50
        Bakso       0.98      0.97      0.97        59
Coto Makassar       0.90      0.90      0.90        52
    Gado Gado       0.96      0.92      0.94        52
        Gudeg       0.90      0.98      0.94        48
  Nasi Goreng       0.95      0.95      0.95        58
       Pempek       0.98      0.98      0.98        53
        Rawon       0.92      0.90      0.91        49
      Rendang       0.89      0.86      0.88        49
  Sate Madura       0.91      0.98      0.95        53
  Sate Padang       1.00      0.93      0.96        54
         Soto       0.98      0.93      0.96        58

     accur

In [17]:
print()
print("=" * 50)
print("TAHAP 2: Membuka Stage Terakhir Swin")
print("=" * 50)
model.enable_stages([3])
optimizer = optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=LR_S2, weight_decay=1e-4)
for epoch in range(EPOCHS_S2):
    train_loss = train_one_epoch(model, train_loader, optimizer, criterion)
    val_loss, f1, report = evaluate(model, val_loader, criterion)
    print(f"Epoch [{epoch + 1}/{EPOCHS_S2}] | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Macro F1: {f1:.4f}")
    print("--- Laporan Klasifikasi Validasi ---")
    print(report)
    if f1 > best_f1:
        best_f1 = f1
        torch.save(model.state_dict(), BEST_MODEL_PATH)
        print(f"✅ Model disimpan (F1 terbaik baru: {best_f1:.4f})")


TAHAP 2: Membuka Stage Terakhir Swin


Train:   0%|          | 0/815 [00:00<?, ?it/s]

/tmp/ipython-input-845850214.py:49: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast_ctx():


Val:   0%|          | 0/100 [00:00<?, ?it/s]

/tmp/ipython-input-845850214.py:72: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast_ctx():


...

Train:   0%|          | 0/815 [00:00<?, ?it/s]

/tmp/ipython-input-845850214.py:49: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast_ctx():


Val:   0%|          | 0/100 [00:00<?, ?it/s]

/tmp/ipython-input-845850214.py:72: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast_ctx():


Epoch [3/3] | Train Loss: 0.1632 | Val Loss: 0.8694 | Macro F1: 0.9110
--- Laporan Klasifikasi Validasi ---
               precision    recall  f1-score   support

   Ayam Bakar       0.81      0.93      0.86        55
  Ayam Betutu       0.85      0.86      0.85        51
  Ayam Goreng       0.86      0.70      0.78        54
     Ayam Pop       0.92      0.96      0.94        50
        Bakso       0.95      1.00      0.98        59
Coto Makassar       0.90      0.90      0.90        52
    Gado Gado       0.94      0.94      0.94        52
        Gudeg       0.92      0.98      0.95        48
  Nasi Goreng       0.98      0.86      0.92        58
       Pempek       0.96      0.94      0.95        53
        Rawon       0.92      0.92      0.92        49
      Rendang       0.88      0.90      0.89        49
  Sate Madura       0.83      0.98      0.90        53
  Sate Padang       1.00      0.85      0.92        54
         Soto       0.98      0.95      0.96        58

     accur

In [18]:
print()
print("=" * 50)
print("TAHAP 3: Fine-tuning Seluruh Backbone")
print("=" * 50)
model.enable_all()
optimizer = optim.AdamW(model.parameters(), weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.OneCycleLR(
    optimizer,
    max_lr=LR_S3,
    epochs=EPOCHS_S3,
    steps_per_epoch=len(train_loader),
)
for epoch in range(EPOCHS_S3):
    train_loss = train_one_epoch(model, train_loader, optimizer, criterion, scheduler)
    val_loss, f1, report = evaluate(model, val_loader, criterion)
    print(f"Epoch [{epoch + 1}/{EPOCHS_S3}] | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Macro F1: {f1:.4f}")
    print("--- Laporan Klasifikasi Validasi ---")
    print(report)
    if f1 > best_f1:
        best_f1 = f1
        torch.save(model.state_dict(), BEST_MODEL_PATH)
        print(f"✅ Model disimpan (F1 terbaik baru: {best_f1:.4f})")

print()
print("=" * 50)
print("TRAINING SELESAI")
print("=" * 50)
print(f"Model terbaik tersimpan di: {BEST_MODEL_PATH} dengan F1 {best_f1:.4f}")


TAHAP 3: Fine-tuning Seluruh Backbone


Train:   0%|          | 0/815 [00:00<?, ?it/s]

/tmp/ipython-input-845850214.py:49: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast_ctx():


Val:   0%|          | 0/100 [00:00<?, ?it/s]

/tmp/ipython-input-845850214.py:72: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast_ctx():


...

Train:   0%|          | 0/815 [00:00<?, ?it/s]

/tmp/ipython-input-845850214.py:49: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast_ctx():


Val:   0%|          | 0/100 [00:00<?, ?it/s]

/tmp/ipython-input-845850214.py:72: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast_ctx():


Epoch [6/6] | Train Loss: 0.0150 | Val Loss: 1.0306 | Macro F1: 0.9227
--- Laporan Klasifikasi Validasi ---
               precision    recall  f1-score   support

   Ayam Bakar       0.89      0.89      0.89        55
  Ayam Betutu       0.89      0.82      0.86        51
  Ayam Goreng       0.84      0.87      0.85        54
     Ayam Pop       0.92      0.98      0.95        50
        Bakso       0.94      1.00      0.97        59
Coto Makassar       0.90      0.88      0.89        52
    Gado Gado       0.94      0.96      0.95        52
        Gudeg       0.94      0.94      0.94        48
  Nasi Goreng       0.93      0.97      0.95        58
       Pempek       0.98      0.94      0.96        53
        Rawon       0.92      0.94      0.93        49
      Rendang       0.90      0.90      0.90        49
  Sate Madura       0.96      0.92      0.94        53
  Sate Padang       0.94      0.94      0.94        54
         Soto       0.94      0.88      0.91        58

     accur

### Catatan Beban Pelatihan yang Diringankan
- Resolusi input turun dari 224→192 piksel (\~26.6% lebih sedikit piksel).
- Batch size turun dari 16→8 (50% lebih ringan per langkah).
- Jumlah epoch kini 2/3/6 untuk setiap stage (turun 33%, 40%, dan 50%).


In [19]:
###################################################################################
### BLOK C: FINAL INFERENCE & SUBMISSION (Single Swin Transformer) ###
###################################################################################

print("\n" + "#" * 80)
print("### MEMULAI BLOK C: INFERENCE & SUBMISSION (Swin Transformer) ###")
print("#" * 80 + "\n")

model_swin = SingleSwinModel(num_classes=len(label_mapping)).to(DEVICE)
MODEL_PATH = BEST_MODEL_PATH
print(f"Memuat bobot dari: {MODEL_PATH}")
model_swin.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
model_swin.eval()

final_test_transform = transforms.Compose([
    CLAHETransform(),
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),
])

test_df = convert_path_to_df(TEST_DIR, is_test=True)
test_dataset = TestDataset(test_df, "Filepath", transform=final_test_transform)
test_loader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS,
    persistent_workers=NUM_WORKERS > 0,
)

reverse_label_mapping = {v: k for k, v in label_mapping.items()}

submission_data = []
with torch.no_grad():
    for images, paths in tqdm(test_loader, desc="Inferensi Swin"):
        images = images.to(DEVICE)
        outputs = model_swin(images)
        preds = torch.argmax(outputs, dim=1)
        for img_path, label_idx in zip(paths, preds.cpu().tolist()):
            img_id = os.path.splitext(os.path.basename(img_path))[0]
            submission_data.append({"id": img_id, "style": reverse_label_mapping[label_idx]})

print("\nMenyimpan hasil prediksi ke submission_swin.csv...")
submission_df = pd.DataFrame(submission_data)
submission_df.sort_values(by="id", inplace=True)
submission_df.to_csv("submission_swin.csv", index=False)

print("✅ File submission_swin.csv berhasil dibuat!")
print("\n" + "=" * 50 + "\nPROSES SELESAI (SWIN TRANSFORMER)\n" + "=" * 50)


################################################################################
### MEMULAI BLOK C: INFERENCE & SUBMISSION (Swin Transformer) ###
################################################################################

Memuat bobot dari: best_swin_model.pth


Inferensi Swin:   0%|          | 0/258 [00:00<?, ?it/s]


Menyimpan hasil prediksi ke submission_swin.csv...
✅ File submission_swin.csv berhasil dibuat!

PROSES SELESAI (SWIN TRANSFORMER)


In [22]:
import pandas as pd

# Muat kembali file submission
submission_df = pd.read_csv("submission_swin.csv")

# Konversi kolom 'id' ke integer untuk menghilangkan leading zeros
submission_df["ID"] = submission_df["id"].astype(int)

# Konversi kolom 'style' menjadi 'label'
submission_df['label'] = submission_df['style']

# Drop column lama
submission_df = submission_df.drop(columns=['id', 'style'])

# Simpan kembali file submission yang sudah diperbaiki
submission_df.to_csv("submission_swin.csv", index=False)

print("✅ Kolom 'id' di submission_swin.csv telah diperbaiki dan disimpan ulang!")

# Tampilkan beberapa baris pertama untuk verifikasi
print("Beberapa baris pertama setelah perbaikan:")
display(submission_df.head())

✅ Kolom 'id' di submission_swin.csv telah diperbaiki dan disimpan ulang!
Beberapa baris pertama setelah perbaikan:


,ID,label
0,1,Sate Padang
1,2,Nasi Goreng
2,3,Ayam Goreng
3,4,Rendang
4,5,Ayam Goreng
